In [1]:
import os
import subprocess
import numpy as np
import cv2
import shutil

In [16]:
dirlist = subprocess.check_output(["find", "/run/user/1000/gvfs/dav:host=nextcloud.kamaro-engineering.de,ssl=true,user=jbarthel,prefix=%2Fremote.php%2Fdav/files/jbarthel/Datenverarbeitung/Datensätze/Löwenzahn/Plastik Modell/Bierdosen 🍺"])

In [18]:
images = [e for e in dirlist.decode('utf-8').splitlines() if e[-3:].lower() in ["jpg","png"]]

In [19]:
names = [f.lower().replace(":","_").replace(" ","_").replace("/markiert","").split("/")[-1] for f in images]

In [21]:
filenames_labels = dict()
filenames_images = dict()

for name, image in zip(names, images):
    if name.endswith("jpg"):
        filenames_images[name[:-4]] = image
    elif name.endswith("png"):
        filenames_labels[name[:-4]] = image
    else:
        raise ValueError("Invalid filename "+name+" for image "+image)

In [22]:
keys = list(filenames_labels.keys())

In [24]:
MARKER_DANDELION = np.array([255,255,0])
MARKER_ALCOHOLIC = np.array([0,255,255])

def load_label(name):
    raw = cv2.imread(filenames_labels[name])[:,:,::-1]
    mask_dandelion = (np.abs(raw - MARKER_DANDELION).mean(axis=-1) < 20 ).astype(np.uint8)
    mask_alcoholic = (np.abs(raw - MARKER_ALCOHOLIC).mean(axis=-1) < 5 ).astype(np.uint8)
    mask_none = np.ones_like(mask_dandelion) - mask_dandelion - mask_alcoholic
    return np.stack([mask_dandelion,mask_alcoholic, mask_none], axis=-1) * 255

In [25]:
for key in keys:
    out_image = key + ".jpg"
    out_label = key + ".png"
    
    if not os.path.isfile(out_image):
        print("copying image", out_image)
        shutil.copyfile(filenames_images[key], out_image)
    else:
        print("skipping image ", out_image)
        
    if not os.path.isfile(out_label):
        print("copying label", out_label)
        label = load_label(key)
        cv2.imwrite(out_label, label)
    else:
        print("skipping label", out_label)

copying image img_2022-05-31t19_06_54.322520.jpg
copying label img_2022-05-31t19_06_54.322520.png
copying image img_2022-05-31t19_06_56.248453.jpg
copying label img_2022-05-31t19_06_56.248453.png
copying image img_2022-05-31t19_07_01.883374.jpg
copying label img_2022-05-31t19_07_01.883374.png
copying image img_2022-05-31t19_07_07.682860.jpg
copying label img_2022-05-31t19_07_07.682860.png
copying image img_2022-05-31t19_07_16.381704.jpg
copying label img_2022-05-31t19_07_16.381704.png
copying image img_2022-05-31t19_07_22.582158.jpg
copying label img_2022-05-31t19_07_22.582158.png
copying image img_2022-05-31t19_07_24.048444.jpg
copying label img_2022-05-31t19_07_24.048444.png
copying image img_2022-05-31t19_07_30.582820.jpg
copying label img_2022-05-31t19_07_30.582820.png
copying image img_2022-05-31t19_07_32.415211.jpg
copying label img_2022-05-31t19_07_32.415211.png
copying image img_2022-05-31t19_07_36.652982.jpg
copying label img_2022-05-31t19_07_36.652982.png
